# Organizations

Generate a Graph for Private Organizations

### Imports

Into utils are stored constants for namespaces, vocabularies and common functions

In [ ]:
# Set autoreload for imported packages

%load_ext autoreload
%autoreload 2


In [ ]:
# Import utils
import os
import sys

rootFolder = os.path.abspath(
    os.path.dirname(
        os.path.dirname(
            os.path.abspath("__file__"))))

sys.path.append(rootFolder)

In [ ]:
from utils import *

In [ ]:
# Imports other required packages

import pandas as pd

from rdflib import Literal

from rdflib.namespace import XSD

from alive_progress import alive_bar

In [ ]:
# Import OntoPiA packages

from ontopia_py import ConceptScheme, saveGraph
from ontopia_py.ns import *
from ontopia_py.clv import *
from ontopia_py.cov import *
from ontopia_py.sm import *

In [ ]:
config = getConfig("../conf.ini")

BASE_URL = config.get("API", "base_url")

## Graph creation

In [ ]:
g = createGraph()

In [ ]:
# Create a ConceptScheme
ORGANIZATIONS_DATA = ConceptScheme(COV_DATA)

# Set the properties
ORGANIZATIONS_DATA.label = [
    Literal("Imprese", lang="it"),
    Literal("Companies", lang="en")
]
ORGANIZATIONS_DATA.creator = [ONTO_AUTHOR]

# And add to graph
ORGANIZATIONS_DATA.addToGraph(g)


In [ ]:
# Get the data

datasetID = config.get("ORGANIZATIONS", "dataset")

# Accomodation facilities
organizations = getOpenData(
    BASE_URL, datasetID, config.get("ORGANIZATIONS", "organizations"), dtype={'IVA': str, 'CODICE_FISCALE': str})
organizations = organizations.set_index(["IVA"])


In [ ]:
# Create emails, phones and websites (to avoid repetitions)

allPecs = pd.DataFrame(organizations["PEC"]).dropna(
).drop_duplicates().set_index(["PEC"])
for pecAddress, _ in allPecs.iterrows():
    pec = Email(
        id="pec-" + genNameForID(pecAddress),
        baseUri=COV_DATA,
        dataset=ORGANIZATIONS_DATA,
        titles=[Literal(pecAddress, datatype=XSD.string)]
    )

    pec.hasEmailType = EmailType(id="041", baseUri=EROGATION_CHANNELS)
    pec.emailAddress = Literal("mailto:" + pecAddress, datatype=XSD.anyURI)

    pec.addToGraph(g, isTopConcept=False)


In [ ]:
with alive_bar(len(organizations), dual_line=True, title='🏭 Organizations') as bar:
    for vatCode, organizationInfo in organizations.iterrows():
        denominazione = standardizeName(organizationInfo["DENOMINAZIONE"])

        bar.text = f'-> Adding {denominazione}'

        codsAteco = organizationInfo["COD_ATECO"]

        taxCode = organizationInfo["CODICE_FISCALE"]
        reaCode = organizationInfo["REA"]

        codTipologia = organizationInfo["COD_TIPOLOGIA"]
        pecAddress = organizationInfo["PEC"]

        startActivityDate = organizationInfo["INIZIO_ATTIVITA"]
        endActivityDate = organizationInfo["FINE_ATTIVITA"]

        progrNazionale = organizationInfo["PROGR_NAZIONALE"]
        progrCivico = organizationInfo["PROGR_CIVICO"] if not pd.isna(
            organizationInfo["PROGR_CIVICO"]) else "snc"

        legalStatus = LegalStatus(
            id=codTipologia.replace(".", ""),
            baseUri=ORG_LEGAL_STATUS
        )

        # Create organization
        organization = PrivateOrganization(
            id="org-" + str(vatCode),
            baseUri=COV_DATA,
            dataset=ORGANIZATIONS_DATA,
            titles=[Literal(denominazione, datatype=XSD.string)]
        )

        # Add attributes to organization
        organization.legalName = [Literal(denominazione, datatype=XSD.string)]
        organization.VATnumber = Literal(vatCode, datatype=XSD.string)
        organization.taxCode = Literal(taxCode, datatype=XSD.string)
        organization.REANumber = Literal(reaCode, datatype=XSD.string)
        organization.hasLegalStatus = legalStatus

        if not pd.isna(startActivityDate):
            organization.startingActivityDate = Literal(
                startActivityDate, datatype=XSD.date)

        # Get address reference
        if not pd.isna(progrNazionale):
            address = Address(
                id="ad-{}-{}".format(progrNazionale, progrCivico),
                baseUri=ANNCSU
            )

            organization.hasPrimaryAddress = address

        organization.hasPrivateOrgActivityType = []
        for codAteco in codsAteco.split(","):
            atecoType = PrivateOrgActivityType(
                id=codAteco.replace(".", ""),
                baseUri=ORG_ATECO
            )
            organization.hasPrivateOrgActivityType.append(atecoType)

        # Create online contact points
        onlineContactPoint = OnlineContactPoint(
            id="ocp-" + str(vatCode),
            baseUri=COV_DATA,
            dataset=ORGANIZATIONS_DATA,
            titles=[
                Literal("Informazioni di contatto per " +
                        denominazione, lang="it"),
                Literal("Contact information for " + denominazione, lang="en"),
            ]
        )

        if not pd.isna(pecAddress):
            pec = Email(
                id="pec-" + genNameForID(pecAddress),
                baseUri=COV_DATA
            )
            onlineContactPoint.hasCertifiedEmail = [pec]

            organization.hasOnlineContactPoint = onlineContactPoint

            onlineContactPoint.addToGraph(g, isTopConcept=False)

        organization.addToGraph(g, isTopConcept=True)

        bar()


In [ ]:
# Save graph
saveGraph(g, "organizations")